In [1]:
from __future__ import print_function, division
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
import numpy as np

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
#from tensorflow.keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import sys
import os



Using TensorFlow backend.
G:\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
G:\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
G:\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
G:\Anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:526: Fu

In [2]:
class GAN():
    def __init__(self, filename):
        self.line_len = 0
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.predict_text = []
        #self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.img_shape = (self.line_len, 1)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        self.prepare_text_data(filename)
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    # Подготовка текстовых данных: удаление функциональных символов в тексте
    def prepare_text_data(self, string):
        fin = open(string, 'rb')
        self.lines = []

        for line in fin:
            line = line.strip().lower()
            line = line.decode("ascii","ignore")
            if len(line)==0:
                continue
            self.lines.append(line)
        fin.close()
        text = " ".join(self.lines)
        specs = "',]\"{?-+_;$*[&)~|=/#}%<:>`!.()1234567890"
       
        for sc in specs:
            text = text.replace(sc, "")
        
   # Подготовка текстовых данных: создание словаря слов уникальных слов
        words = text.split(' ')
    
        keywords = set([w for w in words])
        nb_words = len(keywords)
        self.words2index = dict((w,i) for i, w in enumerate(keywords))
        self.index2words = dict((i,w) for i, w in enumerate(keywords))
   
   # Подготовка текстовых данных: удаление функциональных символов в массивах строк
        self.lineswords = []
    
        self.clearlines = []
        for l in self.lines:
            for sc in specs:
                l = l.replace(sc, "")
            self.clearlines.append(l)
        
        for l in self.clearlines:
            ls = l.split(' ')
            self.lineswords.append(ls)
            
        #print(self.lineswords)
        #print(self.clearlines)
        
    # Подготовка текстовых данных: удаление функциональных символов в массивах строк    
        self.lineswordsdigit = self.lineswords.copy()
        self.linesDigit = self.clearlines.copy()
    # Подготовка текстовых данных: создание числовых массивов-слов   
#         print(self.clearlines[547])
#         print(self.lines[547])
        for l in range (len(self.clearlines)):
            for w in range(len(self.lineswords[l])):
#                 print((l, w))
                #                 if(self.lineswords[l][w] == 'years,'):
                #                     print('lineswords[l][w]');

                wdigit = self.words2index[self.lineswords[l][w]]
                self.lineswordsdigit[l][w] = wdigit
        
    # Подготовка текстовых данных: заполнение массивов чисел до максимальной длины строки
        maxl = 0
        EMPTY_LINE_SYMBOL = '-1' 
        for l in range(len(self.lineswordsdigit)):
            if len(self.lineswordsdigit[l])>maxl:
                maxl = len(self.lineswordsdigit[l])
        
        print("------------------------------------------------------------------------------------" + str(maxl))
        self.line_len = maxl       
        self.img_shape = (self.line_len, 1)
        for l in range(len(self.lineswordsdigit)):
            while len(self.lineswordsdigit[l])<maxl:
                self.lineswordsdigit[l].append(EMPTY_LINE_SYMBOL)
                    
        
  # Создание блока-генератора

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    # Создание блока-дешифратора

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

   
    def train(self, epochs, batch_size=128, sample_interval=1):

        # Load the dataset
      
        X_train = np.array(self.lineswordsdigit).astype('int32')

        # Rescale -1 to 1
        X_train = ((X_train + 1) / (len(self.words2index)+1)-0.5)*2
        X_train = np.expand_dims(X_train, axis=2)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.uniform(-1, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.uniform(-1, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
#             if epoch % sample_interval == 0:
#                 self.sample_text(epoch)
    


    def sample_text(self):
        l = 10
        noise = np.random.uniform (-1, 1, (l, self.latent_dim))
        gen_txt = self.generator.predict(noise)


        gen_txt = (gen_txt+1)*(len(self.words2index)+1)*0.5-1
        gen_txt=np.rint(gen_txt)
        gen_txt=np.array(gen_txt).astype('int32')
        
        t_text = gen_txt.copy()

      
        for l1 in range(len(gen_txt)):
            self.predict_text.append([])
            self.predict_text[0].append([])
            count = 0
            o_level = 0
            i_level = 0
            
            for w1 in range(len(gen_txt[1])):
                if (gen_txt[l1][w1][0]==-1) or (gen_txt[l1][w1][0] >= len(self.index2words)):
                    t_text = ""    
                    count+=1            
                else:
                    t_text = self.index2words[gen_txt[l1][w1][0]]
                    self.predict_text[o_level][i_level].append(t_text)
                    count+=1
                if count == l:
                    i_level+=1
                    count=0
                    self.predict_text.append([])
                    self.predict_text[o_level].append([])
                if i_level ==l:
                    o_level+=1  
                    i_level=0
            
            
        file = open("generics_texts/"+filename+"_generic_text.txt", "w")
        for l_2 in self.predict_text:
            file.write(str(l_2) + '\n')
        file.close()





        



In [3]:
models = []
def mass_text_data():
    for root, dirs, files in os.walk("Texts by genres"):
        for f in files:
            models.append(GAN("Texts by genres/"+f))
            models[len(models)-1].train(10)
            models[len(models)-1].sample_text()
            


In [4]:
mass_text_data()

------------------------------------------------------------------------------------3176
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 3176)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1626624   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 1,758,209
Trainable params: 1,758,209
N

G:\Anaconda3\envs\test\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.340861, acc.: 44.92%] [G loss: 1.168566]
1 [D loss: 0.240631, acc.: 93.75%] [G loss: 1.076888]
2 [D loss: 0.270760, acc.: 91.02%] [G loss: 1.029288]
3 [D loss: 0.306828, acc.: 86.72%] [G loss: 0.992646]
4 [D loss: 0.317484, acc.: 83.59%] [G loss: 0.968873]
5 [D loss: 0.293040, acc.: 86.72%] [G loss: 1.009239]
6 [D loss: 0.281132, acc.: 89.84%] [G loss: 1.029619]
7 [D loss: 0.273683, acc.: 88.67%] [G loss: 1.036252]
8 [D loss: 0.261608, acc.: 92.58%] [G loss: 1.112981]
9 [D loss: 0.259941, acc.: 92.58%] [G loss: 1.159182]


IndexError: list index out of range

In [ ]:
# gan = GAN()
# # gan.prepare_text_data("3Ksentcs.txt")
# gan.train(10)

In [ ]:
# noise = np.random.uniform(-1, 1, (10, gan.latent_dim))
# gen_txt = gan.generator.predict(noise)

# gen_txt = (gen_txt+1)*(len(gan.words2index)+1)*0.5-1
# gen_txt=np.rint(gen_txt)
# gen_txt=np.array(gen_txt).astype('int32')

# print(np.min(gen_txt))
# print(np.max(gen_txt))
# print(len(gan.index2words))

# for l1 in range(len(gen_txt)):
#     for w1 in range(len(gen_txt[1])):        
#         t_text = gan.index2words[gen_txt[l1][w1][0]]
#         gan.predict_text = t_text             

In [ ]:
#X_train = np.array(gan.lineswordsdigit).astype('int32')


In [ ]:
#X_train.shape

In [ ]:
# # Rescale 0 to 1
# X_train = (X_train + 1) / (len(gan.words2index)+1)
# X_train = np.expand_dims(X_train, axis=2)

In [ ]:
#X_train.shape

In [ ]:
#gan.img_shape